# RAG USING JAVELIN WITH COMMENTS

## Install Required Dependencies

In [2]:
!pip install langchain chromadb javelin_sdk

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
INFO: pip is looking at multiple versions of javelin-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached javelin_sdk-0.2.18-py3-none-any.whl.metadata (3.5 kB)
  Using cached javelin_sdk-0.2.17-py3-none-any.whl.metadata (3.4 kB)
  Using cached javelin_sdk-0.2.16-py3-none-any.whl.metadata (3.4 kB)
  Using cached javelin_sdk-0.2.15-py3-none-any.whl.metadata (3.4 kB)
  Using cached javelin_sdk-0.2.14-py3-none-any.whl.metadata (3.4 kB)
  Using cached javelin_sdk-0.2.13-py3-none-any.whl.metadata (3.4 kB)
  Using cached javelin_sdk-0.2.12-py3-none-any.whl.metadata (3.4 kB)
INFO: pip is still looking at multiple versions of javelin-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached javelin_sdk-0.2.11-py3-none-any.whl.metadata (3.3 kB)
  Using cached javelin_sdk-0.2.10-py3-none-any.whl.metadata (3.3 kB)
  Using cached

In [3]:
!pip install -qU langchain-text-splitters



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


### Import Required Libraries

In [4]:
import os
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from javelin_sdk import JavelinClient, JavelinConfig, Route

## Setting Up Api Key

In [5]:
llm_api_key = os.environ["OPENAI_API_KEY"] = ""

javelin_api_key = os.environ["JAVELIN_API_KEY"] = ""


### Initialize Javelin Client

In [6]:
config = JavelinConfig(
    base_url=os.getenv("JAVELIN_BASE_URL"),
    javelin_api_key=os.getenv("JAVELIN_API_KEY"),
    llm_api_key=os.getenv("OPENAI_API_KEY"),
)
client = JavelinClient(config)

In [7]:
# from javelin_sdk import Route

# # Embeddings route
# embeddings_route = Route(
#     name="embeddings_route",
#     type="embedding",
#     models=[
#         {
#             "name": "text-embedding-ada-002",
#             "enabled": True,
#             "provider": "openai",
#             "suffix": "/v1/embeddings",
#         }
#     ],
#     config={
#         "archive": True,
#         "organization": "myusers",
#         "retries": 3,
#         "rate_limit": 7,
#     },
# )
# client.create_route(embeddings_route)

# # Completions route
# completions_route = Route(
#     name="completions_route_2",
#     type="chat",
#     models=[
#         {
#             "name": "gpt-3.5-turbo",
#             "enabled": True,
#             "provider": "openai",
#             "suffix": "/v1/chat/completions",
#         }
#     ],
#     config={
#         "archive": True,
#         "organization": "myusers",
#         "retries": 3,
#         "rate_limit": 7,
#     },
# )
# client.create_route(completions_route)


### Define Custom Embedding Model for Javelin

#### The default JavelinAIGatewayEmbeddings resulted in an AttributeError when using Chroma.
#### The issue: 'dict' object has no attribute 'dict' during embedding function execution.

In [8]:
from langchain_community.embeddings import JavelinAIGatewayEmbeddings
from typing import List

class CustomJavelinAIGatewayEmbeddings(JavelinAIGatewayEmbeddings):
    def _query(self, texts: List[str]) -> List[List[float]]:
        results = []
        for txt in texts:
            try:
                resp = self.client.query_route(self.route, query_body={"input": txt})
                
                # Log the response for debugging
                # print(f"Response for text '{txt}': {resp}")
                
                # Directly extract the embeddings from 'data'
                embeddings_chunk = resp.get("data", [])
                if embeddings_chunk:
                    for item in embeddings_chunk:
                        if "embedding" in item:
                            results.append(item["embedding"])
                else:
                    raise ValueError(f"No embeddings returned for text: {txt}")
            except Exception as e:
                raise ValueError(f"Error in embedding query: {e}")
        return results


# Use the custom subclass
embedding_model_custom = CustomJavelinAIGatewayEmbeddings(
    client=client,
    route="embeddings_route",
)

## Simple inializing javelin model

In [9]:
javelin_embedding_headers = {
    "x-api-key": javelin_api_key,
    "x-javelin-route": "embeddings_route"
}

embedding_model = OpenAIEmbeddings(
    openai_api_base="https://api-dev.javelin.live/v1/query",
    openai_api_key=llm_api_key,
    model_kwargs={"extra_headers": javelin_embedding_headers},
)

In [10]:
# Sample text
sample_text = """
Artificial Intelligence (AI) is a rapidly advancing technology that is transforming industries and societies across the globe. In recent years, the implementation of AI models has revolutionized sectors such as healthcare, automotive, finance, and entertainment, among many others. These technologies enable machines to simulate human-like cognitive functions, such as problem-solving, learning, and decision-making, with unprecedented accuracy and efficiency. AI models leverage vast amounts of data to provide insights, automate complex tasks, and enhance productivity in ways that were once considered unimaginable. As AI continues to evolve, it is reshaping the workforce, influencing economic trends, and even altering the way people interact with technology on a day-to-day basis.

In the healthcare industry, AI applications have the potential to revolutionize patient care, diagnosis, and treatment. Machine learning algorithms are being used to analyze medical data, such as medical images, patient records, and clinical trial results, to assist doctors in making more accurate and timely diagnoses. For example, AI-powered tools can detect early signs of diseases such as cancer, heart conditions, and neurological disorders, improving patient outcomes by enabling early intervention. Moreover, AI has become a key player in personalized medicine, where treatment plans are tailored to individual patients based on genetic data, lifestyle factors, and medical history. This level of precision is helping to optimize the efficacy of treatments and reduce the risk of adverse reactions.

The automotive industry is another sector experiencing a major transformation thanks to AI technology. Autonomous vehicles, also known as self-driving cars, are perhaps the most notable example of AI's impact on this industry. By utilizing machine learning, computer vision, and sensor technologies, these vehicles are capable of navigating roads, detecting obstacles, and making real-time decisions with minimal human intervention. Self-driving cars have the potential to reduce accidents caused by human error, decrease traffic congestion, and improve fuel efficiency. However, the widespread adoption of autonomous vehicles presents numerous challenges, such as regulatory hurdles, public acceptance, and the development of safe, reliable AI systems that can handle the complexities of real-world driving environments.

In the finance sector, AI has revolutionized the way banks and financial institutions analyze and process large volumes of data. AI models are now used for fraud detection, credit scoring, algorithmic trading, and risk management. These systems can detect unusual patterns in financial transactions, identify potential fraudulent activity, and alert security teams in real time. In addition, AI-powered chatbots and virtual assistants are improving customer service by offering instant, personalized responses to client queries. Furthermore, AI-driven predictive analytics are helping investment firms to make smarter decisions by analyzing market trends and forecasting future movements with a level of precision that was once unattainable.

The entertainment industry has also benefited from the integration of AI technologies. Streaming platforms such as Netflix, Spotify, and YouTube use machine learning algorithms to recommend personalized content to users based on their viewing or listening history. These AI-driven recommendation engines not only enhance user experience but also contribute to increased engagement and revenue for these platforms. Additionally, AI is playing a significant role in content creation. Generative AI models, such as OpenAI's GPT-3, can generate text, music, and even visual art based on user input. This opens up new possibilities for artists, writers, and musicians to explore creative avenues and generate novel content quickly and efficiently. For example, AI has been used to compose music, write scripts, and generate visual effects for movies, significantly speeding up the creative process while offering new ways to collaborate with machines.

One of the most exciting and rapidly growing subsets of AI is Generative AI. Unlike traditional AI systems that are designed to recognize patterns and make predictions based on existing data, Generative AI creates entirely new content. This can include generating human-like text, realistic images, and even video content from simple prompts or inputs. For example, GPT-3, a state-of-the-art language model developed by OpenAI, can generate coherent and contextually appropriate text based on a given prompt. It can write essays, poetry, and even code, all with a high degree of fluency and coherence. Similarly, Generative AI models such as DALL·E can create original images based on textual descriptions, allowing users to generate unique visual content without the need for traditional graphic design skills. This has the potential to disrupt various creative industries, such as advertising, marketing, and design, by providing a powerful tool for rapid content generation.

Despite its many advantages, the widespread adoption of AI also presents challenges and risks that need to be addressed. One of the most significant concerns surrounding AI is the potential for job displacement. As AI systems become more capable of performing tasks traditionally done by humans, such as data entry, customer service, and even creative work, there is a growing concern about the impact on employment. While AI has the potential to create new jobs in fields such as data science, machine learning engineering, and AI ethics, it may also lead to job losses in certain sectors. As a result, there is an increasing need for policies and strategies to retrain workers and ensure that they have the skills necessary to thrive in an AI-driven economy.

Another concern is the ethical implications of AI decision-making. AI models are only as good as the data they are trained on, and if that data is biased or incomplete, the resulting decisions made by AI systems can perpetuate existing inequalities. For example, facial recognition systems have been found to be less accurate at identifying people of color, raising concerns about the fairness and accountability of these technologies. There is also the issue of AI transparency, as many machine learning models, particularly deep learning models, operate as "black boxes," making it difficult to understand how they arrive at their decisions. This lack of transparency can undermine trust in AI systems and make it challenging to hold them accountable when things go wrong.

Furthermore, the use of AI in sensitive areas such as surveillance, law enforcement, and military applications raises important questions about privacy and security. As AI technologies become more advanced, there is a risk that they could be used to infringe on individuals' rights and freedoms. For instance, the use of AI-powered surveillance systems to track people's movements and behaviors raises concerns about the erosion of privacy and civil liberties. Similarly, the development of AI-driven weapons and autonomous military systems presents a range of ethical and strategic challenges that need to be carefully considered.

In conclusion, AI is a transformative technology with the potential to revolutionize various industries and improve the quality of life for people around the world. However, as AI continues to evolve, it is important to address the challenges and risks associated with its adoption. This includes ensuring that AI systems are developed in an ethical and responsible manner, addressing potential job displacement, and promoting transparency and fairness in AI decision-making. By doing so, we can unlock the full potential of AI and ensure that its benefits are realized in a way that benefits society as a whole.
"""

documents = [Document(page_content=sample_text)]

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)


In [11]:
print(split_docs)

[Document(metadata={}, page_content='Artificial Intelligence (AI) is a rapidly advancing technology that is transforming industries and societies across the globe. In recent years, the implementation of AI models has revolutionized'), Document(metadata={}, page_content='implementation of AI models has revolutionized sectors such as healthcare, automotive, finance, and entertainment, among many others. These technologies enable machines to simulate human-like'), Document(metadata={}, page_content='enable machines to simulate human-like cognitive functions, such as problem-solving, learning, and decision-making, with unprecedented accuracy and efficiency. AI models leverage vast amounts of data'), Document(metadata={}, page_content='AI models leverage vast amounts of data to provide insights, automate complex tasks, and enhance productivity in ways that were once considered unimaginable. As AI continues to evolve, it is'), Document(metadata={}, page_content='unimaginable. As AI continues

In [12]:
vector_store = Chroma(persist_directory="chroma_store_custom", embedding_function=embedding_model_custom)

/var/folders/z_/93w3rhm91913vgvg7hxgnf9r0000gn/T/ipykernel_83978/2499654517.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory="chroma_store_custom", embedding_function=embedding_model_custom)


In [13]:
vector_store.add_documents(split_docs)

['6546a2a7-427a-4fa3-9df1-d583c4632a81',
 '9799d004-2e20-4385-8567-6e8f57303839',
 '940e69c7-b69c-4696-bc4b-9ca558913232',
 '6ca68275-fb07-4599-9831-f000f379f88d',
 'f439170b-643d-47b6-b11e-cd74a7bbbdfe',
 'ef67810e-52e1-446f-a4c7-39eb31f60319',
 'b437a022-c6af-4474-bcb0-baccb1839233',
 'f7d6c8fc-7cd1-47e6-b8b6-6cf557934ef2',
 'a65b40c1-2267-4832-96ca-e6184179e45a',
 '0213cc59-8129-4458-a299-4cebe2dbb918',
 '1f182987-22a2-4d7f-abd1-875f5c18e58f',
 '993f8c97-e771-44b8-8ec8-5b4de3a45b8c',
 '20bb7463-0a38-492f-bc26-8a6aec5b6614',
 '9af08e05-fca9-4849-9cf4-04b338d9aef7',
 'e101863c-1b17-4299-8928-91490c1f87c9',
 '52e5ad4e-b1e9-4573-9cfc-a98d9b23b8f1',
 'b46dacf9-eba2-4f2c-8fbf-1daa7026811a',
 'b47ea260-cb80-450d-8ec7-1343eb800706',
 '6dd5db45-8212-4347-9b57-374a1b3be717',
 '84bdb2b7-cb00-4df3-a47c-8a7f77817703',
 'b38fc3f2-177d-4369-85f4-69ce0d14a1b7',
 '3a5729e8-9c3a-4da9-bc6d-a7955d5ce53e',
 '428326ad-b828-45d7-b0d5-5af11c6f28c0',
 '66c446b8-7849-4190-873e-91a308e5d179',
 '9bdef75a-11fd-

In [14]:
vector_store = Chroma(persist_directory="chroma_store", embedding_function=embedding_model)

In [15]:
vector_store.add_documents(split_docs)

['49ec94df-7caf-4ac2-9a28-41131dba8f84',
 '80a6ea54-aa27-457a-a490-cb0587c79178',
 '4415cbde-f402-46aa-ae66-a8f2217981b9',
 'c1e66f3b-8ac4-4245-b3fc-a7f8f9e44ab5',
 'bd186fb2-a895-4c19-99d3-5eabfb026a29',
 '00c77085-033f-4158-98a0-da8499b2a5d2',
 '35371d8f-c3c3-46be-91b7-c8980bf80c97',
 '801f9147-aa98-43f1-9d69-49205e09d42c',
 '406702d1-6c7b-4db6-97d7-f2bc3c7624c6',
 '3d6cb01f-eb8a-4963-badb-4b04866f7cdf',
 'c31d1001-d7ed-49bf-a9bf-2e692ef31b71',
 'fb62c030-14a3-4cdd-8abb-0de658e9c0eb',
 '183f448e-dcea-4cfa-b3e0-20f12475d16c',
 'd0294c36-7317-4d9e-8a6b-a456fea19314',
 '18b48642-edf9-4a56-a836-6b1201efeaa1',
 'a4951eeb-8260-491c-b98e-480da357a763',
 '21b6158b-2d07-40a0-9a70-1c3a34e4c4be',
 '1753a4a5-00ad-45f8-ab12-12a4b3dff92d',
 '8c1ed840-63c9-4780-bdaa-00c209727e85',
 'f8c6e367-f147-4e19-bda3-b806f4c54f40',
 '321c8ac2-90dd-4337-aaed-67349f5ad5f5',
 '7e62cd93-d5e3-4aeb-9c7b-2def102cf437',
 '35341ee5-1bbc-4a66-b32a-c1d69672f171',
 '2da66a7b-4370-4652-9e07-70b6cbfd944d',
 'c98f5a82-0288-

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

## Retrieving query without chunks

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os


javelin_headers = {
    "x-api-key": javelin_api_key,
    "x-javelin-route": "completions_route_2"
}

llm = ChatOpenAI(
    openai_api_base="https://api-dev.javelin.live/v1/query",  # Set Javelin's API base URL for query
    openai_api_key=llm_api_key,
    model_kwargs={
        "extra_headers": javelin_headers
    },
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Hello, you are a helpful scientific assistant. Based on the provided documents, answer the user's query."),
    ("user", "{input}"),
    ("assistant", "Here is the relevant context from the documents:\n{documents}")
])

output_parser = StrOutputParser()

# Define the function to retrieve relevant documents
def retrieve_relevant_docs(query):
    relevant_documents = retriever.get_relevant_documents(query)
    # Combine documents into a string
    documents = "\n".join([doc.page_content for doc in relevant_documents])
    print("Documents i get>>",documents)
    return documents

# Define the function to get the final response
def get_final_response(query):
    documents = retrieve_relevant_docs(query)
    
    # Pass the query and documents to the prompt template
    chain = prompt | llm | output_parser
    
    # Create input for the chain
    response = chain.invoke({
        "input": query,
        "documents": documents
    })
    
    return response

# Example query
query = "Do ai provides biased responses?"

# Get the final response
final_answer = get_final_response(query)
print(final_answer)


/var/folders/z_/93w3rhm91913vgvg7hxgnf9r0000gn/T/ipykernel_83978/2040932332.py:31: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_documents = retriever.get_relevant_documents(query)


Documents i get>> Another concern is the ethical implications of AI decision-making. AI models are only as good as the data they are trained on, and if that data is biased or incomplete, the resulting decisions made
how they arrive at their decisions. This lack of transparency can undermine trust in AI systems and make it challenging to hold them accountable when things go wrong.
AI can provide biased responses if the data that it is trained on is biased or incomplete. This can lead to unfair or inaccurate decisions being made. Transparency in how AI systems arrive at their decisions is crucial to ensure accountability and trust.


## Retrieving Query in Chunks 

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# Setup keys and headers
javelin_api_key = os.getenv('JAVELIN_API_KEY')
llm_api_key = os.getenv("OPENAI_API_KEY")
javelin_headers = {
    "x-api-key": javelin_api_key,
    "x-javelin-route": "completions_route_2"
}

# Define the LLM
llm = ChatOpenAI(
    openai_api_base="https://api-dev.javelin.live/v1/query",
    openai_api_key=llm_api_key,
    model_kwargs={
        "extra_headers": javelin_headers
    },
)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Hello, you are a helpful scientific assistant. Based on the provided documents, answer the user's query."),
    ("user", "{input}"),
    ("assistant", "Here is the relevant context from the documents:\n{documents}")
])

output_parser = StrOutputParser()

# Define the chunking function using RecursiveCharacterTextSplitter
def chunk_text(text, chunk_size=1000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

# Define the function to retrieve relevant documents
def retrieve_relevant_docs(query):
    # Chunk the query
    query_chunks = chunk_text(query)
    documents = []

    for chunk in query_chunks:
        relevant_documents = retriever.get_relevant_documents(chunk)
        documents += [doc.page_content for doc in relevant_documents]
    
    # Join all document chunks into one string for prompt
    documents = "\n".join(documents)
    print("Documents I get>>", documents)
    return documents

# Define the function to get the final response
def get_final_response(query):
    documents = retrieve_relevant_docs(query)
    
    # Pass the query and documents to the prompt template
    chain = prompt | llm | output_parser
    
    # Create input for the chain
    response = chain.invoke({
        "input": query,
        "documents": documents
    })
    
    return response

# Example query
query = "Do ai provides biased responses?"

# Get the final response
final_answer = get_final_response(query)


Documents I get>> Another concern is the ethical implications of AI decision-making. AI models are only as good as the data they are trained on, and if that data is biased or incomplete, the resulting decisions made
how they arrive at their decisions. This lack of transparency can undermine trust in AI systems and make it challenging to hold them accountable when things go wrong.


In [19]:
final_answer

'AI can provide biased responses if the data it is trained on is biased or incomplete. It is essential to ensure that AI systems are trained on diverse and representative data to reduce bias in their responses. Additionally, the lack of transparency in AI decision-making can make it challenging to hold these systems accountable for biased responses.'